# How Computers Work

To write good software, you have to understand the platform you software runs on.

Let's learn how a computer works! Basically all computers when decomposed look like this:

![](computer_cross-section.gif)

We can reorganize this into a diagram of how things are connected:

![](computer_layout.jpg)

Various systems (hard disks, RAM, internet connections, mouse/keyboard, etc.) feed in **data** and **instructions** to the CPU for it to process, which it will then output back to one of the systems (internet, RAM, screen, etc.)

Most computer programs end up in an **assembly language** before being fed to the processor as instructions and data to apply the instructions on. It looks like this:

![](dcache_icache.png)

We can even see this for ourselves! Some tools like https://godbolt.org/ help you see the assembly directly from code in "native" languages (programming languages like C, C++, Rust, D and Go that are compiled directly to computer assembly).

### Binary Files

You can also see this if you try to open some computer files that are in a "binary" format in a text editor. The 1's and 0's try to get interpreted as text and you see garbage instead. for a computer, instructions are also data!

If you somehow point the instructions to the wrong data, all sorts of weird thing can end up happening. For instance, old video games have famous *glitches* which descend from this fact:

![](pokemon_glitch_city.png)

# Binary Data types

Numbers in in computers have to be represented as series of `1`s and `0`s. This is done using a **binary encoding**:

![](binary_integer.png)

The first bit adds 0 or 1. The second adds 0 or 2, the third 0 or 4, etc. in powers of 2. Notice `1 + 2 = 3` so the first two bits cover all the numbers below the third bit. This is always true, so all numbers below $2^{n bits}$ are possible.

There are **signed** and **unsigned** numbers, the first reserve a bit for the negative sign, the other use all the bits for positive numbers starting at 0.

# Floats

FLoating point numbers are more complex. They allocate bits to various parts to try to best represent all possible values:

![](floating_point.png)

Note that floating point numbers are **inexact by nature**. So you get surprising behaviors like:

In [4]:
print(0.1 + 0.2)
0.3 == (0.1 + 0.2)

0.30000000000000004


False

# Text

Back in the 1970s all computer scientists were American so they figured we only needed the english latin alphabet and a few additional symbols packed into a byte (8 bits):

![](ascii_table.png)

That's the last time anyone agreed on how to represent text until the 2010s. Since a lot of people want text for different things than english, of which there are more than 255 symbols, they tried UTF-16 and UTF-32 but ended up with [UTF-8](https://www.youtube.com/watch?v=MijmeoH9LT4) which is quite a genious solution to the problem.

# The Memory Hierarchy 

The reason computers are fast is because the basic operations happen billions of times per second. The hard part is feeding in the data for the operations.

You can't have a lot of data stored in a fast way. So we end up with a hierarchy of successfully faster pieces of memory.

![](memory_hierarchy.png)

Accessing each other one is orders of magnitude slower. While the CPU acts in less than one **nanosecond**, it can take 1/5 of a **millisecond** to do a roundtrip to RAM and back

Python is mainly slow because everything has to do round trips to RAM everywhere. Moreover, the language is **interpreted** instead of being compiled (so each command is read, parsed, and executed every time)

In [12]:
import numpy as np
def loop_fetch_python(arr):
    res = 0
    for i in range(1000):
        res += arr[i]
    return res

a = np.random.rand(10_000_000)
loop_fetch_evil(a) # pre-compile the benchmark function
%timeit loop_fetch_evil(a)

39.5 µs ± 624 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


here a lot of the slowness comes from the python variables being accessed over and over.

This would be even slower if we used a python `list` because the memory of the array would be fragmented.

In [11]:
from numba import jit

@jit(nopython=True)
def loop_fetch(arr):
    res = 0
    for i in range(1000):
        res += arr[i]
    return res

a = np.random.rand(10_000_000)

loop_fetch(a) # pre-compile the benchmark function

%timeit loop_fetch(a)

1.47 µs ± 4.95 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The numba directive just says to compile the code to a fast binary (instead of an interpreted python code)

The version above is simple -- on every loop we pull the next number. Since the next place in memory is likely to be in the cache, it's very fast.

The version below is **evil**.

On every iteration of the loop, we jump to a random place in the big array and hence we miss the CPU's cache

In [7]:
@jit(nopython=True)
def loop_fetch_evil(arr):
    res = 0
    for i in range(1000):
        res += arr[np.random.randint(arr.size)]
    return res

a = np.random.rand(10_000_000)

loop_fetch_evil(a) # pre-compile the benchmark function

%timeit loop_fetch_evil(a)

38.5 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
# Now it fits in the CPU cache, it'll be fast again
a = np.random.rand(100)
loop_fetch_evil(a) # pre-compile the benchmark function
%timeit loop_fetch_evil(a)

7.58 µs ± 71.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
